In [6]:
import os
import csv
import numpy as np
from scipy import ndimage as ndi
from skimage import io, img_as_float
from skimage.filters import gabor_kernel
from PIL import Image
from scipy.fftpack import dct as scipy_dct

# Définir le chemin vers le dossier Datasets
base_dir = os.path.expanduser("~/Desktop/Datasets")

# Définir les noms des sous-dossiers
sub_dirs = ["NORMAL", "PNEUMONIA"]

# Créer les filtres Gabor
kernels = []
for theta in range(4):
    theta = theta / 4.0 * np.pi
    for sigma in (1, 3):
        for frequency in (0.05, 0.25):
            kernel = np.real(gabor_kernel(frequency, theta=theta, sigma_x=sigma, sigma_y=sigma))
            kernels.append(kernel)

# Fonction pour calculer les caractéristiques d'une image
def compute_feats(image, kernels):
    feats = np.zeros((len(kernels), 2), dtype=np.double)
    for k, kernel in enumerate(kernels):
        filtered = ndi.convolve(image, kernel, mode='wrap')
        feats[k, 0] = filtered.mean()
        feats[k, 1] = filtered.var()
    return feats

def compute_dct(image):
    if isinstance(image, np.ndarray):
        img = Image.fromarray(image)  # Convert the ndarray to a PIL Image
    else:
        img = Image.open(image)  # Open the image if it's a file path or file-like object
    numpydata = np.asarray(img)
    return scipy_dct(numpydata)


# Chemin du fichier CSV
csv_file_path = os.path.expanduser("~/Desktop/Datasets_features1.csv")

# Créer le fichier CSV et ajouter les en-têtes
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Numéro d'image", "Caractéristique", "Étiquette"])

    # Parcourir chaque sous-dossier pour traiter chaque image
    for sub_dir in sub_dirs:
        folder_path = os.path.join(base_dir, sub_dir)
        
        if os.path.exists(folder_path):
            # Parcourir toutes les images dans le dossier
            for filename in os.listdir(folder_path):
                if filename.endswith(('.png', '.jpg', '.jpeg')):
                    # Charger l'image en niveaux de gris
                    image_path = os.path.join(folder_path, filename)
                    image = img_as_float(io.imread(image_path, as_gray=True))
                    
                    # Calculer le vecteur de caractéristiques
                    feats = compute_feats(image, kernels)
                    dct = compute_dct(image)
                    
                    # Extraire le numéro de l'image et le label
                    print(f"Image file '{filename}' was filtered successfully")
                    numero_image = filename.split('_')[-1].split('.')[0]  # ex: "48" de "PNEUMONIA_48.jpg"
                    label = filename.split('_')[0]  # ex: "PNEUMONIA"
                    
                    # Convertir les caractéristiques en chaîne de caractères
                    feats_str = str(feats.tolist())
                    dct_str = str(dct.tolist())

                    # Écrire la ligne dans le fichier CSV
                    writer.writerow([numero_image, [feats_str, dct_str], label])

print(f"Les caractéristiques des images ont été enregistrées dans '{csv_file_path}'.")

Image file 'NORMAL_22.jpg' was filtered successfully
Image file 'NORMAL_21.jpg' was filtered successfully
Image file 'NORMAL_8.jpg' was filtered successfully
Image file 'NORMAL_9.jpg' was filtered successfully
Image file 'NORMAL_20.jpg' was filtered successfully
Image file 'NORMAL_18.jpg' was filtered successfully
Image file 'NORMAL_19.jpg' was filtered successfully
Image file 'NORMAL_17.jpg' was filtered successfully
Image file 'NORMAL_2.jpg' was filtered successfully
Image file 'NORMAL_3.jpg' was filtered successfully
Image file 'NORMAL_16.jpg' was filtered successfully
Image file 'NORMAL_14.jpg' was filtered successfully
Image file 'NORMAL_1.jpg' was filtered successfully
Image file 'NORMAL_0.jpg' was filtered successfully
Image file 'NORMAL_15.jpg' was filtered successfully
Image file 'NORMAL_11.jpg' was filtered successfully
Image file 'NORMAL_4.jpg' was filtered successfully
Image file 'NORMAL_5.jpg' was filtered successfully
Image file 'NORMAL_10.jpg' was filtered successfully
I